In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! which python

/home/local_arnab/miniconda3/envs/fact/bin/python


In [3]:
import sys
sys.path.append('..')

import torch
torch.__version__, torch.version.cuda

('1.13.1+cu117', '11.7')

In [4]:
from src import models, data, lens, functional
from src.utils import experiment_utils

import logging
from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

In [7]:
device = "cuda:0"
mt = models.load_model("mamba-3b", device=device, fp16=False)
# mt = models.load_model("gptj", device=device, fp16=True)

2024-02-10 18:05:00 src.models INFO     loading state-spaces/mamba-2.8b-slimpj (device=cuda:0, fp16=False)
2024-02-10 18:05:00 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b-slimpj/resolve/main/config.json HTTP/1.1" 200 0
{'d_model': 2560, 'n_layer': 64, 'vocab_size': 50277, 'ssm_cfg': {}, 'rms_norm': True, 'residual_in_fp32': True, 'fused_add_norm': False, 'pad_vocab_size_multiple': 8}
============> Block.__init__() | norm_cls=functools.partial(<class 'mamba.mamba_ssm.models.mixer_seq_simple.RMSNorm'>, eps=1e-05, device=None, dtype=None) | dim=2560<============


TypeError: RMSNorm.__init__() got an unexpected keyword argument 'dtype'

In [6]:
# import baukit

# mt.model.config.fused_add_norm = False

# for block_path in models.determine_layer_paths(mt):
#     block = baukit.get_module(mt.model, block_path)
#     setattr(block, "fused_add_norm", False)

In [7]:
mt.model.backbone.layers[0].fused_add_norm, mt.model.config.fused_add_norm

(False, False)

In [8]:
mt.model.config

MambaConfig(d_model=2560, n_layer=64, vocab_size=50277, ssm_cfg={}, rms_norm=True, residual_in_fp32=True, fused_add_norm=False, pad_vocab_size_multiple=8)

In [9]:
from src.utils.typing import Mamba

isinstance(mt.model, Mamba)

True

In [10]:
"mamba" in str(type(mt.model))

True

In [11]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt, 
    prompt = mt.tokenizer.eos_token + " The capital of {} is".format("France"),
    # prompt = mt.tokenizer.eos_token + " The superlative of {} is".format("good"),
)

self.is_mamba=True | self.is_mamba_fast=True
>>>>>>>>>>>>>>>> Block.forward() | self.fused_add_norm=False <<<<<<<<<<<<<<<<<<<
<class 'mamba_ssm.ops.triton.layernorm.RMSNorm'>
>>>>>>>>>>>>>>>> Block.forward() | self.fused_add_norm=False <<<<<<<<<<<<<<<<<<<
<class 'mamba_ssm.ops.triton.layernorm.RMSNorm'>
>>>>>>>>>>>>>>>> Block.forward() | self.fused_add_norm=False <<<<<<<<<<<<<<<<<<<
<class 'mamba_ssm.ops.triton.layernorm.RMSNorm'>
>>>>>>>>>>>>>>>> Block.forward() | self.fused_add_norm=False <<<<<<<<<<<<<<<<<<<
<class 'mamba_ssm.ops.triton.layernorm.RMSNorm'>
>>>>>>>>>>>>>>>> Block.forward() | self.fused_add_norm=False <<<<<<<<<<<<<<<<<<<
<class 'mamba_ssm.ops.triton.layernorm.RMSNorm'>
>>>>>>>>>>>>>>>> Block.forward() | self.fused_add_norm=False <<<<<<<<<<<<<<<<<<<
<class 'mamba_ssm.ops.triton.layernorm.RMSNorm'>
>>>>>>>>>>>>>>>> Block.forward() | self.fused_add_norm=False <<<<<<<<<<<<<<<<<<<
<class 'mamba_ssm.ops.triton.layernorm.RMSNorm'>
>>>>>>>>>>>>>>>> Block.forward() | self.fused

[[PredictedToken(token=' Paris', prob=0.34679484367370605),
  PredictedToken(token=' a', prob=0.07820745557546616),
  PredictedToken(token=' the', prob=0.059068549424409866),
  PredictedToken(token=' located', prob=0.052434541285037994),
  PredictedToken(token=' also', prob=0.02528112567961216)]]

In [12]:
mt.model

MambaLMHeadModel(
  (backbone): MixerModel(
    (embedding): Embedding(50280, 2560)
    (layers): ModuleList(
      (0): Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
          (act): SiLU()
          (x_proj): Linear(in_features=5120, out_features=192, bias=False)
          (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
          (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
        )
        (norm): RMSNorm()
      )
      (1): Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
          (act): SiLU()
          (x_proj): Linear(in_features=5120, out_features=192, bias=False)
          (dt_proj): Linear(in_features=160, out_features=512

In [12]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt, 
    prompt = mt.tokenizer.eos_token + " The capital of {} is".format("France"),
    # prompt = mt.tokenizer.eos_token + " The superlative of {} is".format("good"),
)

self.is_mamba=True | self.is_mamba_fast=True


[[PredictedToken(token=' Paris', prob=0.34679484367370605),
  PredictedToken(token=' a', prob=0.07820745557546616),
  PredictedToken(token=' the', prob=0.059068549424409866),
  PredictedToken(token=' located', prob=0.052434541285037994),
  PredictedToken(token=' also', prob=0.02528112567961216)]]

In [13]:
dataset = data.load_dataset()

# relation_names = [r.name for r in dataset.relations]
# relation_options = Menu(choices = relation_names, value = relation_names)
# show(relation_options) # !caution: tested in a juputer-notebook. baukit visualizations are not supported in vscode.

2024-02-09 09:50:51 src.data DEBUG    no paths provided, using default data dir: /home/local_arnab/Codes/relations/notebooks/../data
2024-02-09 09:50:51 src.data DEBUG    /home/local_arnab/Codes/relations/notebooks/../data is directory, globbing for json files...
2024-02-09 09:50:51 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/characteristic_gender.json
2024-02-09 09:50:51 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/degree_gender.json
2024-02-09 09:50:51 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_birthplace.json
2024-02-09 09:50:51 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_gender.json
2024-02-09 09:50:51 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_religion.json
2024-02-09 09:50:51 src.data DEBUG    found relation 

In [14]:
relation_name = "country capital city"
relation = dataset.filter(relation_names=[relation_name])[0]
print(f"{relation.name} -- {len(relation.samples)} samples")
print("------------------------------------------------------")

experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
train, test = relation.split(5)
print("\n".join([sample.__str__() for sample in train.samples]))

2024-02-09 09:50:52 src.data DEBUG    filtering to only relations: ['country capital city']
country capital city -- 24 samples
------------------------------------------------------
2024-02-09 09:50:52 src.utils.experiment_utils INFO     setting all seeds to 12345
China -> Beijing
Japan -> Tokyo
Italy -> Rome
Brazil -> Bras\u00edlia
Turkey -> Ankara


In [15]:
################### hparams ###################
layer = 20
beta = 8
###############################################

In [16]:
from src.operators import JacobianIclMeanEstimator
experiment_utils.set_seed(12345)

estimator = JacobianIclMeanEstimator(
    mt = mt, 
    h_layer = layer,
    beta = beta
)
operator = estimator(
    relation.set(
        samples=train.samples, 
    )
)

2024-02-09 09:50:54 src.utils.experiment_utils INFO     setting all seeds to 12345
2024-02-09 09:50:54 src.operators WARNING  relation has > 1 prompt_templates, will use first (The capital city of {} is)
2024-02-09 09:50:54 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of China is
2024-02-09 09:50:54 src.operators DEBUG    note that subject=China, h_index=26


self.is_mamba=True | self.is_mamba_fast=True
h.shape=torch.Size([2560]) | h.requires_grad=True
self.is_mamba=True | self.is_mamba_fast=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
mt.name

'mamba-3b'

In [14]:
import os
root_path = "../results/cached_o1_approxes"
path = os.path.join(
    root_path,
    mt.name,
    relation_name.lower().replace(" ", "_"),
    str(layer),
)

approxes = []
for cached_file in os.listdir(path):
    approx = functional.load_cached_linear_operator(file_path = os.path.join(path, cached_file))
    approxes.append(approx)


In [15]:
weight = torch.stack([approx.weight for approx in approxes]).mean(dim=0)
bias = torch.stack([approx.bias for approx in approxes]).mean(dim=0)

prompt_template = relation.prompt_templates[0]

prompt_template_icl = functional.make_prompt(
    mt = mt,
    prompt_template=prompt_template,
    subject="{}",
    examples = [
        data.RelationSample.from_dict(approx.metadata["sample"]) 
        for approx in approxes
    ][:min(3, len(approxes))],
)

print(prompt_template_icl)

<|endoftext|> The capital city of China is Beijing
The capital city of Nigeria is Abuja
The capital city of Colombia is Bogot\u00e1
The capital city of {} is


In [16]:
from src.operators import LinearRelationOperator
operator = LinearRelationOperator(
    mt = mt,
    weight = weight,
    bias = bias,
    h_layer = approxes[0].h_layer,
    z_layer = approxes[0].z_layer,
    beta = 5,
    prompt_template = prompt_template_icl,
)

In [17]:
from src.lens import logit_lens
from src import models

# logit_lens(mt = mt, h = operator.metadata["Jh"][0].to(models.determine_device(mt)) + operator.bias)

In [18]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt, 
    prompt = mt.tokenizer.eos_token + " The capital of {} is".format("France"),
    # prompt = mt.tokenizer.eos_token + " The superlative of {} is".format("good"),
)

[[PredictedToken(token=' Paris', prob=0.34679311513900757),
  PredictedToken(token=' a', prob=0.07820795476436615),
  PredictedToken(token=' the', prob=0.05906893312931061),
  PredictedToken(token=' located', prob=0.05243447795510292),
  PredictedToken(token=' also', prob=0.0252813883125782)]]

# Checking $faithfulness$

In [19]:
test = functional.filter_relation_samples_based_on_provided_fewshots(
    mt=mt, test_relation=test, prompt_template=operator.prompt_template, batch_size=4
)

2024-02-07 14:44:50 src.functional DEBUG    filtering for knowns using prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Nigeria is Abuja
The capital city of Colombia is Bogot\u00e1
The capital city of {} is"


2024-02-07 14:44:50 src.functional DEBUG    sample.subject='South Korea', sample.object='Seoul', predicted=' Seoul' (p=0.969), known=(✓)
2024-02-07 14:44:50 src.functional DEBUG    sample.subject='Colombia', sample.object='Bogot\\u00e1', predicted=' Bog' (p=0.739), known=(✓)
2024-02-07 14:44:50 src.functional DEBUG    sample.subject='Saudi Arabia', sample.object='Riyadh', predicted=' R' (p=0.807), known=(✓)
2024-02-07 14:44:50 src.functional DEBUG    sample.subject='France', sample.object='Paris', predicted=' Paris' (p=0.976), known=(✓)
2024-02-07 14:44:50 src.functional DEBUG    sample.subject='Mexico', sample.object='Mexico City', predicted=' Mexico' (p=0.956), known=(✓)
2024-02-07 14:44:50 src.functional DEBUG    sample.subject='Pakistan', sample.object='Islamabad', predicted=' Islam' (p=0.941), known=(✓)
2024-02-07 14:44:50 src.functional DEBUG    sample.subject='Argentina', sample.object='Buenos Aires', predicted=' Buenos' (p=0.947), known=(✓)
2024-02-07 14:44:50 src.functional DE

In [20]:
sample = [s for s in test.samples if s.subject == "France"][0]
print(sample)
operator(subject = sample.subject).predictions

France -> Paris
2024-02-07 14:44:57 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Nigeria is Abuja
The capital city of Colombia is Bogot\u00e1
The capital city of France is"


[PredictedToken(token=' Paris', prob=0.8592612743377686),
 PredictedToken(token=' ', prob=0.056149572134017944),
 PredictedToken(token=' Capital', prob=0.035243552178144455),
 PredictedToken(token='\n', prob=0.0163775235414505),
 PredictedToken(token=' -', prob=0.004834211431443691)]

In [21]:
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject],
    h_layer= operator.h_layer,
)
h = hs_and_zs.h_by_subj[sample.subject]

## Approximating LM computation $F$ as an affine transformation

### $$ F(\mathbf{s}, c_r) \approx \beta \, W_r \mathbf{s} + b_r $$

In [22]:
z = 5 * (operator.weight @ h) + operator.bias

lens.logit_lens(
    mt = mt,
    h = z,
    get_proba = True
)

([(' Paris', 0.859),
  (' ', 0.056),
  (' Capital', 0.035),
  ('\n', 0.016),
  (' -', 0.005),
  (' (', 0.005),
  (' capital', 0.004),
  (' in', 0.003),
  (' Cairo', 0.002),
  ('...', 0.002)],
 {})

In [23]:
correct = 0
wrong = 0
for sample in test.samples:
    predictions = operator(subject = sample.subject).predictions
    known_flag = functional.is_nontrivial_prefix(
        prediction=predictions[0].token, target=sample.object
    )
    print(f"{sample.subject=}, {sample.object=}, ", end="")
    print(f'predicted="{functional.format_whitespace(predictions[0].token)}", (p={predictions[0].prob}), known=({functional.get_tick_marker(known_flag)})')
    
    correct += known_flag
    wrong += not known_flag
    
faithfulness = correct/(correct + wrong)

print("------------------------------------------------------------")
print(f"Faithfulness (@1) = {faithfulness}")
print("------------------------------------------------------------")

2024-02-07 14:45:01 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Nigeria is Abuja
The capital city of Colombia is Bogot\u00e1
The capital city of Argentina is"


sample.subject='Argentina', sample.object='Buenos Aires', predicted="\n", (p=0.20366910099983215), known=(✗)
2024-02-07 14:45:01 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Nigeria is Abuja
The capital city of Colombia is Bogot\u00e1
The capital city of Australia is"
sample.subject='Australia', sample.object='Canberra', predicted=" Capital", (p=0.7717655897140503), known=(✗)
2024-02-07 14:45:01 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Nigeria is Abuja
The capital city of Colombia is Bogot\u00e1
The capital city of Canada is"
sample.subject='Canada', sample.object='Ottawa', predicted=" Capital", (p=0.35436227917671204), known=(✗)
2024-02-07 14:45:01 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Nigeria is Abuja
The capital city of Colombia is Bogot\u00e1
The capita

# $causality$

In [24]:
################### hparams ###################
rank = 100
###############################################

In [25]:
experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
test_targets = functional.random_edit_targets(test.samples)

2024-02-07 14:45:06 src.utils.experiment_utils INFO     setting all seeds to 12345


## setup

In [26]:
source = test.samples[0]
target = test_targets[source]

f"Changing the mapping ({source}) to ({source.subject} -> {target.object})"

'Changing the mapping (Argentina -> Buenos Aires) to (Argentina -> Riyadh)'

### Calculate $\Delta \mathbf{s}$ such that $\mathbf{s} + \Delta \mathbf{s} \approx \mathbf{s}'$

<p align="center">
    <img align="center" src="../causality_crop" style="width:80%;"/>
</p>

Under the relation $r =\, $*plays the instrument*, and given the subject $s =\, $*Miles Davis*, the model will predict $o =\, $*trumpet* **(a)**; and given the subject $s' =\, $*Cat Stevens*, the model will now predict $o' =\, $*guiter* **(b)**. 

If the computation from $\mathbf{s}$ to $\mathbf{o}$ is well-approximated by $operator$ parameterized by $W_r$ and $b_r$ **(c)**, then $\Delta{\mathbf{s}}$ **(d)** should tell us the direction of change from $\mathbf{s}$ to $\mathbf{s}'$. Thus, $\tilde{\mathbf{s}}=\mathbf{s}+\Delta\mathbf{s}$ would be an approximation of $\mathbf{s}'$ and patching $\tilde{\mathbf{s}}$ in place of $\mathbf{s}$ should change the prediction to $o'$ = *guitar* 

In [27]:
def get_delta_s(
    operator, 
    source_subject, 
    target_subject,
    rank = 100,
    fix_latent_norm = None, # if set, will fix the norms of z_source and z_target
):
    w_p_inv = functional.low_rank_pinv(
        matrix = operator.weight,
        rank=rank,
    )
    hs_and_zs = functional.compute_hs_and_zs(
        mt = mt,
        prompt_template = operator.prompt_template,
        subjects = [source_subject, target_subject],
        h_layer= operator.h_layer,
        z_layer=-1,
    )

    z_source = hs_and_zs.z_by_subj[source_subject]
    z_target = hs_and_zs.z_by_subj[target_subject]
    
    z_source *= fix_latent_norm / z_source.norm() if fix_latent_norm is not None else 1.0
    z_target *= z_source.norm() / z_target.norm() if fix_latent_norm is not None else 1.0

    delta_s = w_p_inv @  (z_target.squeeze() - z_source.squeeze())

    return delta_s, hs_and_zs

delta_s, hs_and_zs = get_delta_s(
    operator = operator,
    source_subject = source.subject,
    target_subject = target.subject,
    rank = rank
)

In [28]:
import baukit

def get_intervention(h, int_layer, subj_idx):
    def edit_output(output, layer):
        if(layer != int_layer):
            return output
        functional.untuple(output)[:, subj_idx] = h 
        return output
    return edit_output

prompt = operator.prompt_template.format(source.subject)

h_index, inputs = functional.find_subject_token_index(
    mt=mt,
    prompt=prompt,
    subject=source.subject,
)

h_layer, z_layer = models.determine_layer_paths(model = mt, layers = [layer, -1])

with baukit.TraceDict(
    mt.model, layers = [h_layer, z_layer],
    edit_output=get_intervention(
#         h = hs_and_zs.h_by_subj[source.subject],         # let the computation proceed as usual
        h = hs_and_zs.h_by_subj[source.subject] + delta_s, # replace s with s + delta_s
        int_layer = h_layer, 
        subj_idx = h_index
    )
) as traces:
    outputs = mt(
        input_ids = inputs.input_ids,
        attention_mask = inputs.attention_mask,
    )

logits = outputs.logits[0][-1] if hasattr(outputs, "logits") else outputs[0][-1]
lens.interpret_logits(
    mt = mt, 
    logits = logits, 
    get_proba=True
)

[(' Me', 0.592),
 (' R', 0.181),
 (' Jed', 0.063),
 (' Mak', 0.046),
 (' Med', 0.013),
 (' Dh', 0.012),
 (' Saudi', 0.011),
 (' Dubai', 0.005),
 (' Kuwait', 0.005),
 (' K', 0.004)]

## Measuring causality

In [29]:
from src.editors import LowRankPInvEditor

svd = torch.svd(operator.weight.float())
editor = LowRankPInvEditor(
    lre=operator,
    rank=rank,
    svd=svd,
)

In [30]:
# precomputing latents to speed things up
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject for sample in test.samples],
    h_layer= operator.h_layer,
    z_layer=-1,
    batch_size = 2
)

success = 0
fails = 0

for sample in test.samples:
    target = test_targets.get(sample)
    assert target is not None
    edit_result = editor(
        subject = sample.subject,
        target = target.subject
    )
    
    success_flag = functional.is_nontrivial_prefix(
        prediction=edit_result.predicted_tokens[0].token, target=target.object
    )
    
    print(f"Mapping {sample.subject} -> {target.object} | edit result={edit_result.predicted_tokens[0]} | success=({functional.get_tick_marker(success_flag)})")
    
    success += success_flag
    fails += not success_flag
    
causality = success / (success + fails)

print("------------------------------------------------------------")
print(f"Causality (@1) = {causality}")
print("------------------------------------------------------------")

2024-02-07 14:45:11 src.editors DEBUG    computing low-rank pinv (rel=<|endoftext|> The capital city of China is Beijing
The capital city of Nigeria is Abuja
The capital city of Colombia is Bogot\u00e1
The capital city of {} is)
Mapping Argentina -> Riyadh | edit result=' Me' (p=0.456) | success=(✗)
Mapping Australia -> Buenos Aires | edit result=' Buenos' (p=0.187) | success=(✓)
Mapping Canada -> Abuja | edit result=' Abu' (p=0.435) | success=(✓)
Mapping Chile -> Lima | edit result=' L' (p=0.974) | success=(✓)
Mapping Colombia -> Berlin | edit result=' Bog' (p=0.925) | success=(✗)
Mapping Egypt -> Mexico City | edit result=' Mexico' (p=0.894) | success=(✓)
Mapping France -> Riyadh | edit result=' Me' (p=0.669) | success=(✗)
Mapping Germany -> Cairo | edit result=' Cairo' (p=0.934) | success=(✓)
Mapping India -> Lima | edit result=' L' (p=0.890) | success=(✓)
Mapping Mexico -> Santiago | edit result=' Su' (p=0.205) | success=(✗)
Mapping Nigeria -> Riyadh | edit result=' Abu' (p=0.722) 

In [28]:
from src.functional import save_linear_operator

save_linear_operator(
    approx = operator,
    file_name = "lre_capital",
    path = "cached"
)

In [29]:
operator_loaded = functional.load_cached_linear_operator(mt = mt, file_path = "cached/lre_capital.npz")

## Test

In [ ]:
prompt = mt.tokenizer.eos_token + " Michael Jordan professionally played the sport of"
tokenized = mt.tokenizer(prompt, return_tensors="pt").to(device)
models.determine_layer_paths(mt)[layer]

In [ ]:
import baukit

layer_out = models.determine_layer_paths(mt)[layer]
mixer_out = models.determine_layer_paths(mt)[layer] + ".mixer"
layer_in = models.determine_layer_paths(mt)[layer+1]
final_layer = models.determine_layer_paths(mt)[-1]
final_mixer = models.determine_layer_paths(mt)[-1] + ".mixer"

with baukit.TraceDict(
    module=mt.model,
    layers=[
        layer_out,
        mixer_out,
        layer_in,
        final_mixer,
        final_layer,
        "backbone"
    ],
    retain_input=True,
) as traces:
    output = mt(**tokenized)

In [ ]:
traces["backbone"].output

In [ ]:
traces[final_layer].output

In [ ]:
from torch import nn

class RMSNorm(nn.Module):
    def __init__(self,
                 d_model: int,
                 eps: float = 1e-5):
        super().__init__()
        self.eps = eps
        self.weight = baukit.get_module(mt.model, "backbone.norm_f").weight


    def forward(self, x):
        output = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps) * self.weight

        return output

custom_rms = RMSNorm(d_model = models.determine_hidden_size(mt))
block_output, residual = traces[final_layer].output
backbone_output = custom_rms(block_output + residual)

backbone_output

In [ ]:
torch.allclose(traces[mixer_out].output, traces[layer_out].output[0])

In [ ]:
baukit.get_module(mt.model, "backbone.norm_f").bias

In [ ]:
hasattr(mt.model, "backbone")

In [45]:
n_layers = 32
n_approx = 25
time_per_approx = 7 # in minutes
size_per_approx = 26 # in MB
n_threads = 2


total_time = n_layers * n_approx * time_per_approx
total_time /= n_threads
total_time /= 60

total_size = n_layers * n_approx * size_per_approx
total_size /= 1024

f"Time: {total_time} hours, Size: {total_size} GB"

'Time: 46.666666666666664 hours, Size: 20.3125 GB'

In [47]:
models.determine_layers(mt)

(0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63)

In [49]:
import numpy as np

np.arange(0, 64, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
       34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62])